# Part 1

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from time import sleep
from IPython.display import display
import folium

In [5]:
toronto_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
toronto_soup = BeautifulSoup(toronto_url,'lxml')

In [21]:
table = []
for tr in toronto_soup.find('tbody'):
    for td in tr:
        for element in td:
            if (element != '\n'):
                if (element.find('<')!=-1):
                    tsoup = BeautifulSoup(str(element), 'lxml')
                    element = tsoup.find('a').text
                if "\n" in element:
                    element = element.rstrip('\n')
                table.append(str(element))

In [10]:
for i in range(0, len(table)):
    if(table[i]==table[i-1]):
        table[i-2] = ""
        table[i-1] = ""
        table[i] = ""      
clean = list(filter(None, table))

In [11]:
for i in range(0, len(clean)):      
    if(clean[i]=='Not assigned'):
        clean[i] = clean[i-1]

In [15]:
split = np.array_split(clean,len(clean)/3)
toronto_df = pd.DataFrame(split[1:], columns=split[0])
toronto_df['Neighbourhood'] = toronto_df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x)) 
toronto_df.drop_duplicates(inplace=True)
toronto_df.reset_index(drop=True, inplace=True)

In [16]:
toronto_df.shape
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


# part 2

In [30]:
!wget -O 'Geospatial_data' http://cocl.us/Geospatial_data
print('Download Complete')

--2019-02-28 15:32:09--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-02-28 15:32:09--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-28 15:32:12--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-28 15:32:12--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [33]:
tgeo_df = pd.read_csv('Geospatial_data')

In [34]:
tgeo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
tgeo_df.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [36]:
merge_df = pd.merge(toronto_df,tgeo_df, on='Postcode')

In [37]:
merge_df.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


# Part 3